In [ ]:
import pandas as pd
import numpy as np

In [656]:
HofP=pd.read_json('./HeroesOfPymoli/purchase_data.json')
HofP.head(10)

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59
5,20,Male,10,Sleepwalker,1.73,Tanimnya91
6,20,Male,153,Mercenary Sabre,4.57,Undjaskla97
7,29,Female,169,"Interrogator, Blood Blade of the Queen",3.32,Iathenudil29
8,25,Male,118,"Ghost Reaver, Longsword of Magic",2.77,Sondenasta63
9,31,Male,99,"Expiration, Warscythe Of Lost Worlds",4.53,Hilaerin92


### Player Count

In [ ]:
pd.DataFrame(data={'Total Players': [HofP['SN'].nunique()]})

### Purchasing Analysis (Total)

In [ ]:
keys =['Number of Unique Items','Average Price','Number of Purchases','Total Revenue']
unique_items= HofP['Item ID'].nunique()                  
ave_price = HofP['Price'].mean()
total_purchase = HofP['Item ID'].count()
total_revenue = HofP['Price'].sum()
purchase_ana = {'Number of Unique Items': [unique_items],'Average Price': [ave_price],
                'Number of Purchases': [total_purchase],'Total Revenue': [total_revenue]}
purchase_ana_df = pd.DataFrame(data=purchase_ana, columns=keys)
purchase_ana_df['Average Price'] = purchase_ana_df['Average Price'].map('${:,.2f}'.format)
purchase_ana_df['Total Revenue'] = purchase_ana_df['Total Revenue'].map('${:,.2f}'.format)
purchase_ana_df

### Gender Demographics

In [ ]:
cols=['Percentage of Players',' Total Count ']
genders=[HofP['Gender'].unique()]
gender_grp_counts=pd.DataFrame(HofP.groupby(HofP['Gender'])['SN'].nunique(), 
                               index=genders)
per_plyr=pd.DataFrame(gender_grp_counts['SN']/gender_grp_counts['SN'].sum()*100)
res=per_plyr.set_index(genders).join(gender_grp_counts.set_index(genders),
                                                     lsuffix='0', rsuffix='1')
res['SN0'] = res['SN0'].map('{:,.2f}'.format)
res.rename(columns={'SN0':cols[0],'SN1':cols[1]})

### Purchasing Analysis (Gender)

In [ ]:
HofP['Normalized Totals']=pd.DataFrame(HofP.groupby(['Gender'])['Price']
                                       .apply(lambda x: (x) / x.count()))
pur_ana=pd.DataFrame(HofP.groupby(['Gender']).agg({'Item ID': 'count',
                                                    'Price': 'mean',
                                                   'Normalized Totals':'sum',
                                                     })).rename(columns={'Item ID':'Purchase Count',
                                                                         'Price':'Average Purchase Price'})
pur_ana['Total Purchase Value']= pur_ana['Purchase Count']*pur_ana['Average Purchase Price']
#pur_ana['Normalized Totals']=(HofP.groupby(['Gender'])['Price']
#                                       .apply(lambda x: (x) / x.count())).sum()
pur_ana

In [662]:
pur_ana=gender_grp_counts.join([mean],sort=True)
pur_ana['Total Purchase Value ']=[pur_ana['Price'][x]*pur_ana['SN'][x] for x in range(len(genders[0]))]
pur_ana['Normalized Totals']=[pur_ana['Price'][x]*pur_ana['SN'][x] for x in range(len(genders[0]))]
pur_ana.groupby(['SN'])
pur_ana

def Pyr_count(SN):
    return (len(SN.unique()))
def Perc_Pyr(SN):
    return (len(SN.unique())/HofP['SN'].nunique()*100)

a=HofP.groupby(pd.cut(HofP['Age'] , 
                    bins=np.arange(HofP['Age'].min()-1,
                                   HofP['Age'].max()+8,4), 
                      right=True)).agg({'SN':{Perc_Pyr, Pyr_count}}
                                      )#.rename(columns={'SN':'Total Count'})
a



SN           
         Pyr_count   Perc_Pyr
Age                          
(6, 10]       22.0   3.839442
(10, 14]      20.0   3.490401
(14, 18]      84.0  14.659686
(18, 22]     178.0  31.064572
(22, 26]     153.0  26.701571
(26, 30]      44.0   7.678883
(30, 34]      34.0   5.933682
(34, 38]      25.0   4.363002
(38, 42]      11.0   1.919721
(42, 46]       2.0   0.349040
(46, 50]       NaN        NaN

In [620]:
a.index=[[ "{0}-{1}".format(i+1, i + 4) for i in range(HofP['Age'].min()-1, HofP['Age'].max()+4, 4) ]]

In [653]:
a=pd.DataFrame(a)#.dtypes#keys()#['Total Count']#.map('{:,i}'.format)#'<'+a.index[0].split('-')[1]
type(a)


pandas.core.frame.DataFrame

In [602]:
for a, b in HofP.groupby(pd.cut(HofP['Age'] , 
                    bins=np.arange(HofP['Age'].min()-1,
                                   HofP['Age'].max()+8,4), 
                      right=True)):
    print(a,len(b['SN'].unique()))
    #print((b['SN'].unique()))

(6, 10] 22
(10, 14] 20
(14, 18] 84
(18, 22] 178
(22, 26] 153
(26, 30] 44
(30, 34] 34
(34, 38] 25
(38, 42] 11
(42, 46] 2
(46, 50] 0
